In [1]:
import pandas as pd
import datasist as ds
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier, XGBRFClassifier
import joblib

sns.set_style("dark")

In [2]:
data = pd.read_csv(r"D:\other\Epsilon\Epsilon Training\project 2\my solution\data\part2.csv")
df = pd.DataFrame(data)
df.drop("Unnamed: 0", axis=1, inplace=True)
print(f"Data Shape is : {df.shape}")
df.head()

Data Shape is : (537, 12)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1


In [3]:
ds.structdata.describe(df)

First five data points


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1




Random five data points


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
63,Male,No,0,Graduate,No,3500,0.0,81.0,300.0,1.0,Semiurban,1
98,Male,Yes,0,Not Graduate,No,3572,4114.0,152.0,NaN,0.0,Rural,0
475,Male,Yes,1,Not Graduate,No,2239,2524.0,128.0,360.0,1.0,Urban,1
467,Male,No,1,Graduate,No,11250,0.0,196.0,360.0,NaN,Semiurban,0
343,Male,Yes,0,Graduate,No,2583,2115.0,120.0,360.0,NaN,Urban,1




Last five data points


...

,count,unique,top,freq
Gender,537,2,Male,437
Married,537,2,Yes,349
Education,537,2,Graduate,415
Self_Employed,537,2,No,464
Property_Area,537,3,Semiurban,204




Unique class Count of Categorical features


,Feature,Unique Count
0,Gender,2
1,Married,2
2,Education,2
3,Self_Employed,2
4,Property_Area,3




Missing Values in Data


,features,missing_counts,missing_percent
0,Gender,0,0.0
1,Married,0,0.0
2,Dependents,0,0.0
3,Education,0,0.0
4,Self_Employed,0,0.0
5,ApplicantIncome,0,0.0
6,CoapplicantIncome,0,0.0
7,LoanAmount,19,3.5
8,Loan_Amount_Term,12,2.2
9,Credit_History,41,7.6


In [4]:
df["Dependents"] = df["Dependents"].astype("object")
df["Dependents"]

0      0
1      1
2      0
3      0
4      0
      ..
532    0
533    1
534    1
535    2
536    0
Name: Dependents, Length: 537, dtype: object

### Fill Messing Value by KnnImputer
    - using Pipeline

In [5]:
ds.structdata.display_missing(df)

,features,missing_counts,missing_percent
0,Gender,0,0.0
1,Married,0,0.0
2,Dependents,0,0.0
3,Education,0,0.0
4,Self_Employed,0,0.0
5,ApplicantIncome,0,0.0
6,CoapplicantIncome,0,0.0
7,LoanAmount,19,3.5
8,Loan_Amount_Term,12,2.2
9,Credit_History,41,7.6


In [6]:
df.isna().sum()

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           19
Loan_Amount_Term     12
Credit_History       41
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
cats = df.select_dtypes("object").columns
miss_col = ["LoanAmount", "Loan_Amount_Term", "Credit_History"]

col_trans = ColumnTransformer(

    transformers=[
        ("one", OneHotEncoder(sparse=False , drop="first"), cats),
        ("imputer", KNNImputer(n_neighbors=7), miss_col)
    ], remainder="passthrough"
)

### Use Precision metrics
     - becouse We use precision when we want the prediction of 1 to be as correct as possible 
                 and we use recall when we want our model to spot as many real 1 as possible. 
                 
      - Choosing the correct metric for our model can actually increase its predictive power 
                  and give us a great competitive advantage.

In [8]:
models = {
    "logisr": LogisticRegression(), 
    "knnclass": KNeighborsClassifier(), 
    "dectree": DecisionTreeClassifier(), 
    "svc": SVC(),
    "rfc": RandomForestClassifier(), 
    "xgbc": XGBClassifier(), 
    "xgbrfc": XGBRFClassifier()
}

In [9]:
pipeline = Pipeline(
    
    steps=[
        ("incoder", col_trans), 
        ("scaler", StandardScaler()), 
        ("pca", PCA(0.95)),
        ("dt", models["logisr"])
    ]

)

In [10]:
X = df.drop("Loan_Status" , axis = 1 )
y = df["Loan_Status"]

In [11]:
score = cross_validate(pipeline , X , y , cv = 5 , scoring="accuracy" , return_train_score=True)

In [12]:
score["train_score"].mean()

0.8068043584322654

In [13]:
score["test_score"].mean()

0.8082727587400484

In [14]:
model = models["logisr"]

In [15]:
pipeline = Pipeline(
    
    steps=[
        ("incoder", col_trans), 
        ("scaler", StandardScaler())
    ]

)


for key, value in models.items():
    
    a = tuple([key, value])
    pipeline.steps.append(a)
    score = cross_validate(pipeline , X , y , cv = 5 , scoring="precision" , return_train_score=True)
    pipeline.steps.pop()
    
    print("-" * 100, "\n")
    print(key)
    print("Train_accuracy" , score["train_score"].mean() )
    print("Test_accuracy" , score["test_score"].mean())

    print("-" * 100, "\n")

---------------------------------------------------------------------------------------------------- 

logisr
Train_accuracy 0.7971951661048632
Test_accuracy 0.8015196361309398
---------------------------------------------------------------------------------------------------- 

---------------------------------------------------------------------------------------------------- 

knnclass
Train_accuracy 0.8107289896315486
Test_accuracy 0.7764021850727081
---------------------------------------------------------------------------------------------------- 

---------------------------------------------------------------------------------------------------- 

dectree
Train_accuracy 1.0
Test_accuracy 0.8201098364094722
---------------------------------------------------------------------------------------------------- 

---------------------------------------------------------------------------------------------------- 

svc
Train_accuracy 0.8127721512706456
Test_accuracy 0.793146743177702

### hyperparameter tuning for xgbc, rfc, dectree and logisr algorithms

In [16]:
pipeline = Pipeline(
    
    steps=[
        ("incoder", col_trans), 
        ("scaler", StandardScaler()),
        ("Model", model)
    ]

)

In [17]:
def fit_model(model, params):
    
    pipeline.steps[-1] = tuple(["Model", model])
    
    rs = GridSearchCV(estimator=pipeline, param_grid=params, cv=10, 
                              scoring='precision', n_jobs=-1, verbose=3, return_train_score=True)
    
    rs.fit(X, y)
    
    
    return rs

In [18]:
logisr_params = [
    {'Model__penalty': ['l1', 'l2', 'elasticnet'], 'Model__C': [1, 10, 100, 1000]}
]

logisr_rs = fit_model(models["logisr"], logisr_params)
logisr_rs.best_score_

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   18.3s finished


0.7989629123461115

In [19]:
rfc_params = [
    {'Model__criterion': ['gini', 'entropy'], 'Model__max_depth': range(1, 40), 'Model__n_estimators': range(100, 201, 20)}
]

rfc_rs = fit_model(models["rfc"], rfc_params)
rfc_rs.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 468 candidates, totalling 4680 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   43.2s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 3856 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 4592 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 4680 out of 4680 | elapsed: 12.2min finished


0.8107370029166784

In [20]:
dectree_params = [
    {'Model__criterion': ['gini', 'entropy'], 'Model__max_depth': range(1, 100)}
]

dectree_rs = fit_model(models["dectree"], dectree_params)
dectree_rs.best_score_

Fitting 10 folds for each of 198 candidates, totalling 1980 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   15.1s
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 1980 out of 1980 | elapsed:   31.0s finished


0.8280982976099324

In [21]:
knnclass_params = [
    {'Model__n_neighbors': range(5, 200, 2)}
]

knnclass_rs = fit_model(models["knnclass"], knnclass_params)
knnclass_rs.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 98 candidates, totalling 980 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 980 out of 980 | elapsed:   30.8s finished


0.7819318638813799

In [22]:
xgbc_params = [
    {'Model__criterion': ['gini', 'entropy'], 'Model__max_depth': range(1, 20), 'Model__n_estimators': range(100, 501, 50)}
]

xgbc_rs = fit_model(models["xgbc"], xgbc_params)
xgbc_rs.best_score_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 342 candidates, totalling 3420 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 3420 out of 3420 | elapsed:  5.5min finished


[07:11:20] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "criterion" } are not used.



0.8121653915279838

In [23]:
svc_params = [
    {'Model__C': [1, 10, 100, 1000], 'Model__kernel': ['rbf'], 'Model__gamma': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]},
    {'Model__C': [1, 10, 100, 1000], 'Model__kernel': ['poly'], 'Model__degree': range(2, 20)},
    {'Model__C': [1, 10, 100, 1000], 'Model__kernel': ['sigmoid']}
]

svc_rs = fit_model(models["svc"], svc_params)
svc_rs.best_score_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   31.8s finished


0.8029928227169314

### Save Files

In [24]:
model = dectree_rs.best_estimator_
model

Pipeline(steps=[('incoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Property_Area'],
      dtype='object')),
                                                 ('imputer',
                                                  KNNImputer(n_neighbors=7),
                                                  ['LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History'])])),
                ('scaler', StandardScaler()),
                ('Model', DecisionTreeClassifier(max_depth=73))])

In [25]:
score = cross_validate(dectree_rs.best_estimator_ , X , y , cv = 5 , scoring="precision" , return_train_score=True)

In [26]:
print(f"Train of precision Score is : {score['train_score'].mean()}\n test : {score['test_score'].mean()}")

Train of precision Score is : 1.0
 test : 0.8014074465399016


### Try to Reduce Outliers

In [27]:
dectree_rs.best_estimator_.steps[-1] = ('Model', DecisionTreeClassifier(max_depth=48))

In [28]:
for i in range(1, 62):
    dectree_rs.best_estimator_.steps[-1] = ('Model', DecisionTreeClassifier(max_depth=i))
    score = cross_validate(dectree_rs.best_estimator_ , X , y , cv = 5 , scoring="precision" , return_train_score=True)
    print(f"max_depth = {i}")
    print(f"Train of precision Score is : {score['train_score'].mean()}\n test : {score['test_score'].mean()}")
    print("\n", "-"*100, "\n")

max_depth = 1
Train of precision Score is : 0.7957262868647141
 test : 0.7968607458901928

 ---------------------------------------------------------------------------------------------------- 

max_depth = 2
Train of precision Score is : 0.7961737699953355
 test : 0.7968607458901928

 ---------------------------------------------------------------------------------------------------- 

max_depth = 3
Train of precision Score is : 0.8038475430302284
 test : 0.7984202044540233

 ---------------------------------------------------------------------------------------------------- 

max_depth = 4
Train of precision Score is : 0.8084015409991906
 test : 0.7990928712861785

 ---------------------------------------------------------------------------------------------------- 

max_depth = 5
Train of precision Score is : 0.8397759469903393
 test : 0.7914196835664391

 ---------------------------------------------------------------------------------------------------- 

max_depth = 6
Train of pr

### Reduce max_depth from 48 to 10 to Reduce overfitting

### max_depth = 10
    - Train of precision Score is : 0.9376853652480038
    - test : 0.8112562789915158

In [29]:
dectree_rs.best_estimator_.steps[-1] = ('Model', DecisionTreeClassifier(max_depth=10))

model = dectree_rs.best_estimator_
model.fit(X, y)

Pipeline(steps=[('incoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Property_Area'],
      dtype='object')),
                                                 ('imputer',
                                                  KNNImputer(n_neighbors=7),
                                                  ['LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History'])])),
                ('scaler', StandardScaler()),
                ('Model', DecisionTreeClassifier(max_depth=10))])

In [30]:
Loan_Amount_Term_unique = df["Loan_Amount_Term"].dropna(axis=0).unique()
Loan_Amount_Term_unique

array([360., 120., 240., 180.,  60., 300., 480.,  36.,  84.,  12.])

In [31]:
joblib.dump(model, r"D:\other\Epsilon\Epsilon Training\project 2\my solution\save files\model.h5")
joblib.dump(X.columns, r"D:\other\Epsilon\Epsilon Training\project 2\my solution\save files\input.h5")
joblib.dump(Loan_Amount_Term_unique, r"D:\other\Epsilon\Epsilon Training\project 2\my solution\save files\Loan_Amount_Term.h5")

['D:\\other\\Epsilon\\Epsilon Training\\project 2\\my solution\\save files\\Loan_Amount_Term.h5']